In [1]:
import subprocess
import os
import time
from network_simulator import make_yml, make_network_agents_yml

In [2]:
folder = os.path.join('schema')
name_simulation = 'simulation_tutorial'

In [3]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "neutral"}
    ],

}

In [4]:
INTERVALS = 100
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.05,  # 006
        "prob_tv_spread": 0.05,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.05,  # 003
        "prob_dead": 0.001,  # 001,
        "prob_repost": 0.8,
        "mean_time_connection": 10,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": INTERVALS,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 20,
        "m": 5
    },
    "num_trials": 1
}

In [5]:
# Define your probabilities here
prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.2},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )
for type_i in types:
    for response in responses:
        name_i = f"prob_{type_i}_{response}"
        prob_i = prob_response[type_i][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

In [6]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join(folder, f'{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

FileNotFoundError: [Errno 2] No such file or directory: 'schema\\simulation_tutorial.yml'

In [ ]:
command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 6.0488739013671875


In [ ]:
import os
from network_simulator import get_pivoted_data, get_type_agents
from network_simulator import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from network_simulator import transform_time
from network_simulator import correctness_prompt, correctness_percentage
from anytree import Node, RenderTree, LevelOrderIter
from network_simulator import Post

In [ ]:
from network_simulator.gpt3_5 import send_prompt_openai as send_prompt

In [ ]:
name_simulation = 'simulation_tutorial'

In [ ]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

In [ ]:
attributes = ['cause', 'method', 'response', 'stance', 'repost']

data = get_pivoted_data(analysis_path, sql_table_path, attributes)
dict_agents = get_type_agents(analysis_path, sql_table_path)

In [ ]:
NEWS = 'They dictate preventive detention for Pablo Mackenna after being involved in a traffic accident while intoxicated in Las Condes. '
NEWS_BODY = '''
According to the information being handled, Mackenna crashed an executive taxi on Avenida Presidente Errázuriz and Calle Sánchez Fontecilla, causing serious damage to the other vehicle and leaving one person injured. When performing the breathalyzer, he returned 1.27 grams of alcohol per liter of blood. "We have to look at the conduct of the accused and the way in which he puts the lives of third parties at risk, which actually happened today," said Judge Acevedo. She added that "he hit a taxi driver, that the vehicle is his source of work, therefore, he will be unable to work, in addition to the license he has... he also injured another person." . Likewise, she emphasized that Mackenna crossed the red light, which is why she "committed a traffic violation, in addition to driving while intoxicated." "He will agree to the request of the Prosecutor's Office, and preventive detention will be ordered," said the judge, specifying that an investigation period of 90 days was determined.
'''
SHOW_TREE = True

In [ ]:
root = Post(0, message=NEWS, step=0, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS)

In [ ]:
# 24 hours format
TIME_BEG = "10:00"
TIME_END = "23:00"

In [ ]:
list_nodes = [root]
num_real_messages = 0
for i in range(len(data)):
    aux = data[i]
    owner = int(aux[0])
    step = int(aux[1])
    step = transform_time(step, TIME_BEG, TIME_END)
    id_message = int(aux[2])
    parent = list_nodes[int(aux[3])]
    state = aux[4] #unused
    attr_dict = {}
    for index, attr in enumerate(attributes):
        index +=5
        attr_dict[attr] = aux[index] 

    type_agent = dict_agents[owner]
    
    ### Specific for repost attribute
    if attr_dict["repost"] in ["0", "1"]:
        attr_dict["repost"] = bool(int(attr_dict["repost"]))
    else:
        print(data[i])
        print(attr_dict["repost"])
        raise ValueError("Error repost format")
        
    if not attr_dict["repost"]:
        num_real_messages += 1
    ### 

    message = ''
    news_i = NEWS
    if type_agent == "wise":
        news_i += NEWS_BODY

    list_nodes.append(Post(name=id_message, parent=parent, owner=owner, step=step, message=message, type_agent=type_agent,
                             post_template=POST_TEMPLATE, reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE, news=news_i,
                             **attr_dict
                             )
    )

In [ ]:
from parameters import API_KEY
import openai

client = openai.OpenAI(api_key=API_KEY)

In [ ]:
import time

TEMPERATURE = 1.0
list_prompts = []
i_tot = 0
i = 0
num_llm_errors = 0
MAX_LLM_ERRORS = 15
t = time.time()
t_0 = t

time_list = [0 for _ in range(num_real_messages)]
for node_i in LevelOrderIter(root):
    i_tot += 1
    if node_i == root:
        continue
    elif node_i.repost:
        continue
    i += 1
    instructions, prompt = node_i.get_prompt(language='english', 
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    error_llm = True
    while error_llm:
        try:
            answer = send_prompt(client, instructions, prompt, temp=TEMPERATURE, max_tokens=1000, timeout=40)
            error_llm = False
        except Exception as err:
            print(f"Error encountered (message {i}/{num_real_messages}):", err)
            print("Instrucciones:", instructions)
            print("prompt", prompt)
            time.sleep(2)
            num_llm_errors += 1
            error_llm = True
            if num_llm_errors >= MAX_LLM_ERRORS:
                print("Max Errors reached. Local break run")
                break
    if num_llm_errors >= MAX_LLM_ERRORS:
        print("Max Errors reached. Break run")
        break
    correctness = correctness_percentage(answer)
    while correctness >= 0.2:
        print("Correction")
        correction_prompt = correctness_prompt(node_i.news, answer)
        print(correction_prompt)
        answer = send_prompt(client, instructions, correction_prompt, temp=TEMPERATURE, max_tokens=1000, timeout=40)
        correctness = correctness_percentage(answer)
    
    node_i.set_message(answer)
    list_prompts.append((node_i.name, prompt))
    t_aux = time.time()
    print(f"TIME: {t_aux-t}. Iter tot: {i_tot}. n. message: {i}")
    time_list[i-1] = t
    t = t_aux

print("TOTAL TIME:", t-t_0)

TIME: 1.7036099433898926. Iter tot: 9. n. message: 1
TIME: 2.089984178543091. Iter tot: 10. n. message: 2
TIME: 1.6078288555145264. Iter tot: 11. n. message: 3
TIME: 2.025413751602173. Iter tot: 12. n. message: 4
TIME: 2.027782440185547. Iter tot: 15. n. message: 5
TIME: 1.7999041080474854. Iter tot: 16. n. message: 6
TIME: 2.0287561416625977. Iter tot: 17. n. message: 7
TIME: 2.2945263385772705. Iter tot: 19. n. message: 8
TIME: 2.224000930786133. Iter tot: 20. n. message: 9
TIME: 2.1664443016052246. Iter tot: 22. n. message: 10
TIME: 2.356137275695801. Iter tot: 23. n. message: 11
TIME: 1.7229893207550049. Iter tot: 24. n. message: 12
TIME: 1.5496349334716797. Iter tot: 26. n. message: 13
TIME: 1.613502025604248. Iter tot: 27. n. message: 14
TIME: 2.3039307594299316. Iter tot: 28. n. message: 15
TIME: 1.8274755477905273. Iter tot: 29. n. message: 16
TIME: 1.9301550388336182. Iter tot: 30. n. message: 17
TIME: 2.22218656539917. Iter tot: 31. n. message: 18
TIME: 2.3877642154693604. It

In [ ]:
print(i, "/", num_real_messages)

74 / 74


In [ ]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.name == 0:
            print(f"{pre}NEWS: {node.message}")
            continue

        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        print(f"{pre}{node.owner}<<{node.name}>> (t={node.step})({node.type_agent})({node.parent.name}) {message}")

NEWS: They dictate preventive detention for Pablo Mackenna after being involved in a traffic accident while intoxicated in Las Condes. 
├── 14<<1>> (t=10:22)(wise)(0) repost
│   ├── 10<<5>> (t=11:15)(dumb)(1) repost
│   └── 13<<6>> (t=11:46)(wise)(1) repost
│       ├── 7<<10>> (t=12:11)(herd)(6) repost
│       │   ├── 3<<12>> (t=12:12)(dumb)(10) repost
│       │   ├── 18<<14>> (t=12:45)(wise)(10) 'Seriously, this guy needs to face the consequences of his actions. It's not just about being drunk and causing an accident, but also injuring someone and wrecking a person's livelihood. Locking him up is the right move. 🚓🔒💥'
│       │   ├── 12<<28>> (t=14:22)(herd)(10) 'Finally some justice! It's about time these entitled celebrities face the consequences of their actions. Driving while intoxicated and causing an accident is completely irresponsible. Hopefully, this will serve as a wake-up call for others. 🚔👮‍♀️ #NoMoreCelebrityPrivilege'
│       │   ├── 13<<29>> (t=14:26)(wise)(10) 'Wow, ser